![spreedsheet_finance](spreedsheet_finance.jpg)

Your project is centered around developing a reinforcement learning (RL) simulation for stock trading in Python. This initiative is spearheaded by Quantum Trading, a fictional but ambitious trading firm looking to leverage cutting-edge machine learning techniques to gain a competitive edge in the financial markets. Quantum Trading is a small but highly specialised team of financial analysts, data scientists, and software engineers who are passionate about transforming the way trading decisions are made.

In the fast-paced world of financial markets, staying ahead of the curve is crucial. Traditional trading strategies, while effective, often rely on historical data and predefined rules that may not adapt quickly to changing market conditions. Reinforcement learning, a subfield of machine learning where an agent learns to make decisions by interacting with an environment, offers a promising alternative. It allows the trading algorithms to learn and adapt in real-time, improving their performance as they gain more experience.

By engaging with this project, you will gain valuable insights into the dynamic world of algorithmic trading and enhance your skill set in data science, finance, and machine learning. Remember, the journey of learning and experimentation is as important as the results. Good luck, and may your trading algorithms be ever profitable!

### The Data
The provided data `AAPL.csv` contains historical prices for AAPL (the ticker symbol for Apple Inc) and you will be using this in your model. It has been loaded for you already in the sample code below and contains two columns, described below.

| Column | Description |
|--------|-------------|
|`Date`    | The date corresponding to the closing price              |
|`Close`   | The closing price of the security on the given date      |

_**Disclaimer: This project is for educational purposes only. It is not financial advice, and should not be understood or construed as, financial advice.**_

### Objective

Create and train a Proximal Policy Optimization (PPO) model using the provided environment. Train this model for a `total_timesteps` of **10,000**. Implement the trading logic to make buy, sell, or hold decisions based on the model's predictions.

- Adjust the `cash balance` and `shares_held` based on the action taken (buy/sell) and allow the model to trade a certain percentage of its balance each step.
- Monitor balance changes and store the history in `balance_history` for visualization.
- If shares are still held at the end, sell them to update the final balance.

By the end of this project you should have **two charts** that display the performance of the model:

1. **Chart 1**: Show the stock price with buy/sell actions overlaid on the price.
2. **Chart 2**: Show the cash balance over time.

---

## Questions:

1. **What do you think about the performance of this model?**
2. **Why do you think it has performed the way it has?**
3. **How could the data we have selected impact the performance of the model?**

In [4]:
# pip install gymnasium gym-anytrading stable-baselines3
# Import required packages
# Note that gym-anytrading is a gym environment specific for trading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import Actions
from stable_baselines3 import PPO

# Load the data provided
data = pd.read_csv("AAPL.csv", parse_dates=True, index_col='Date')

In [6]:
data.head()

,Close
Date,
2009-07-09,4.870000
2009-07-10,4.947143
2009-07-13,5.083571
2009-07-14,5.081071
2009-07-15,5.245714


In [8]:
# Set window size (lookback window), start and end index of data
window_size = 10
start_index = window_size
end_index = len(data)

# Create the environment using the pre-built 'stocks-v0', passing in the data loaded above
# window_size is how far the environment can look back at each step
# frame_bound is the bounding box of the environment, the size of our data
env = gym.make('stocks-v0', df=data, window_size=window_size, frame_bound=(start_index, end_index))
print("Observation Space:", env.observation_space)

# Initialise cash balance tracking and other variables
balance = 100000
balance_history = [balance]
shares_held = 0
action_stats = {Actions.Sell: 0, Actions.Buy: 0}
observation, info = env.reset(seed=2024)

Observation Space: Box(-10000000000.0, 10000000000.0, (10, 2), float32)


In [10]:
# Train the PPO model
model = PPO('MlpPolicy', env, verbose=0)
model.learn(total_timesteps=10000)

step = 0
while True:
    action, _states = model.predict(observation)
    current_price = env.unwrapped.prices[env.unwrapped._current_tick]

    observation, reward, terminated, truncated, info = env.step(action)

    trade_amount = balance * 0.10  # Trade with 10% of current balance

    if action == Actions.Buy.value:
        shares_to_buy = trade_amount / current_price
        shares_held += shares_to_buy
        balance -= trade_amount
        print(f"{step}: BUY  {shares_to_buy:.2f} shares at ${current_price:.2f} | Balance: ${balance:.2f}")
    elif action == Actions.Sell.value and shares_held > 0:
        balance += shares_held * current_price
        print(f"{step}: SELL {shares_held:.2f} shares at ${current_price:.2f} | Balance: ${balance:.2f}")
        shares_held = 0
    else:
        print(f"{step}: HOLD | Current price: ${current_price:.2f} | Balance: ${balance:.2f}")

    action_stats[Actions(action)] += 1
    balance_history.append(balance)

    step += 1
    if terminated or truncated:
        break

# If we still hold shares at the end, sell them all
if shares_held > 0:
    balance += shares_held * current_price
    print(f"\nFinal SELL {shares_held:.2f} shares at ${current_price:.2f} | Balance: ${balance:.2f}")
    shares_held = 0

env.close()

print("Action stats:", action_stats)
print(f"Final Balance: ${balance:.2f}")

0: BUY  134.43 shares at $74.39 | Balance: $90000.00
1: SELL 134.43 shares at $75.16 | Balance: $100103.17
2: BUY  131.83 shares at $75.93 | Balance: $90092.86
3: BUY  116.20 shares at $77.53 | Balance: $81083.57
4: SELL 248.03 shares at $78.75 | Balance: $100616.36
5: BUY  129.24 shares at $77.85 | Balance: $90554.73
6: BUY  117.74 shares at $76.91 | Balance: $81499.25
7: BUY  105.32 shares at $77.39 | Balance: $73349.33
8: SELL 352.29 shares at $76.93 | Balance: $100450.40
9: HOLD | Current price: $78.74 | Balance: $100450.40
10: BUY  128.31 shares at $78.29 | Balance: $90405.36
11: BUY  113.28 shares at $79.81 | Balance: $81364.82
12: BUY  102.72 shares at $79.21 | Balance: $73228.34
13: BUY  91.85 shares at $79.72 | Balance: $65905.51
14: BUY  83.23 shares at $79.18 | Balance: $59314.96
15: SELL 519.40 shares at $79.53 | Balance: $100621.27
16: HOLD | Current price: $79.56 | Balance: $100621.27
17: HOLD | Current price: $79.49 | Balance: $100621.27
18: HOLD | Current price: $80.46 

In [ ]:
# Create your two charts below. Note, do not change the fig and ax variable names.

# Create a price chart showing buy/sell actions
fig, ax = plt.subplots()
env.unwrapped.render_all()
ax.set_title("PPO Agent - Trading Actions")
plt.show()

# Create a chart showing cash balance over time
fig2, ax2 = plt.subplots()
ax2.plot(balance_history)
ax2.set_title("PPO Agent - Balance over time")
ax2.set_xlabel("Steps")
ax2.set_ylabel("Balance ($)")

fig2.tight_layout()
plt.show()

Questions/Answers
- What do you think about the performance of this model?
The model has technically been profitable in that the cash balance is higher than what it started with. However, it is very hard to not be profitable given the extreme increase in the price of AAPL over the 15 years of data provided. You would have made significantly more by simply spending all of the initial balance on shares at the starting date, holding the shares, and selling at the ending date. You likely would have made significantly more over that time period by investing your initial balance in any number of asset classes.

- Why do you think it has performed the way it has?
Every transaction that this model makes is the same size. That is why your balance history will have a lot of spikes and drops, as the model buys with 10% of its balance and then sells again instantly with 10% of its balance, making either a small profit or loss. An improvement that could be made here is calculating the position size based on the confidence the model has in its decision, taking a smaller position if less confident and a larger position if more confident. This model also does not account for any transaction costs or slippage which would be a key factor in a real trading scenario.

- How could the data we have selected impact the performance of the model?
This model uses 15 years of data which is a long time. It is very hard to not make money given a stock that has gone from around $4 to over $200! An interesting experiment would be to select some stock data for a company that has not seen the same extreme growth that Apple has seen and observe how the model performs.